In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bert-pretuned/bert_model_bilstm 4/bert_model_bilstm_fold_4.h5
/kaggle/input/bert-pretuned/bert_model_bilstm 2/bert_model_bilstm_fold_0.h5
/kaggle/input/bert-pretuned/bert_model_bilstm 3/bert_model_bilstm_fold_2.h5
/kaggle/input/bert-pretuned/bert_model_bilstm 5/bert_model_bilstm_fold_1.h5
/kaggle/input/bert-pretuned/bert_model_bilstm/bert_model_bilstm_fold_3.h5
/kaggle/input/transformers/transformers-master/.gitignore
/kaggle/input/transformers/transformers-master/hubconf.py
/kaggle/input/transformers/transformers-master/requirements.txt
/kaggle/input/transformers/transformers-master/LICENSE
/kaggle/input/transformers/transformers-master/.coveragerc
/kaggle/input/transformers/transformers-master/MANIFEST.in
/kaggle/input/transformers/transformers-master/requirements-dev.txt
/kaggle/input/transformers/transformers-master/setup.py
/kaggle/input/transformers/transformers-master/README.md
/kaggle/input/transformers/transformers-master/CONTRIBUTING.md
/kaggle/input/transformer

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
from tensorflow.keras.models import load_model

np.set_printoptions(suppress=True)


In [3]:
PATH = '../input/google-quest-challenge/'
BERT_PATH = '../input/bert-uncased/'
tokenizer = tokenization.FullTokenizer('../input/bert-uncased/assets/vocab.txt', True)
MAX_SEQUENCE_LENGTH = 512

In [4]:
df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')

In [5]:
output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])

In [6]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [7]:
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(
            spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data, test_data, batch_size=16, fold=None):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        
        self.batch_size = batch_size
        self.fold = fold
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
        
        rho_val = compute_spearmanr(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None:
            self.model.save_weights(f'bert-base-{fold}-{epoch}.h5py')
        
        self.test_predictions.append(
            self.model.predict(self.test_inputs, batch_size=self.batch_size)
        )

def bert_model():
    
    input_word_ids = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    input_segments = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')
    
    bert_layer = hub.KerasLayer(BERT_PATH, trainable=True)
    
    _, sequence_output = bert_layer([input_word_ids, input_masks, input_segments])
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(768, return_sequences=True))(sequence_output)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(768))(x)
    # x = tf.keras.layers.concatenate([x, input_features])
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(
        inputs=[input_word_ids, input_masks, input_segments], outputs=out)
    
    return model     
        
def train_and_predict(model, train_data, valid_data, test_data, 
                      learning_rate, epochs, batch_size, loss_function, fold):
        
    custom_callback = CustomCallback(
        valid_data=(valid_data[0], valid_data[1]), 
        test_data=test_data,
        batch_size=batch_size,
        fold=None)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=loss_function, optimizer=optimizer)
    model.fit(train_data[0], train_data[1], epochs=epochs, 
              batch_size=batch_size, callbacks=[custom_callback])
    
    return custom_callback

In [8]:
gkf = GroupKFold(n_splits=5).split(X=df_train.question_body, groups=df_train.question_body)

# outputs = compute_output_arrays(df_train, output_categories)
# inputs = compute_input_arays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [9]:
# histories = []
# for fold, (train_idx, valid_idx) in enumerate(gkf):
    
#     # will actually only do 3 folds (out of 5) to manage < 2h
#     K.clear_session()
#     model = bert_model()

#     train_inputs = [inputs[i][train_idx] for i in range(3)]
#     train_outputs = outputs[train_idx]

#     valid_inputs = [inputs[i][valid_idx] for i in range(3)]
#     valid_outputs = outputs[valid_idx]

#     # history contains two lists of valid and test preds respectively:
#     #  [valid_predictions_{fold}, test_predictions_{fold}]
#     history = train_and_predict(model, 
#                       train_data=(train_inputs, train_outputs), 
#                       valid_data=(valid_inputs, valid_outputs),
#                       test_data=test_inputs, 
#                       learning_rate=3e-5, epochs=4, batch_size=8,
#                       loss_function='binary_crossentropy', fold=fold)

#     histories.append(history)

In [10]:
models = []
model_path1 = f'../input/bert-pretuned/bert_model_bilstm/bert_model_bilstm_fold_3.h5'
model_path2 = f'../input/bert-pretuned/bert_model_bilstm 2/bert_model_bilstm_fold_0.h5'
model_path3 = f'../input/bert-pretuned/bert_model_bilstm 3/bert_model_bilstm_fold_2.h5'
model_path4 = f'../input/bert-pretuned/bert_model_bilstm 4/bert_model_bilstm_fold_4.h5'
model_path5 = f'../input/bert-pretuned/bert_model_bilstm 5/bert_model_bilstm_fold_1.h5'
#print(model_path)
model = bert_model()
model.load_weights(model_path1)
models.append(model)

model = bert_model()
model.load_weights(model_path2)
models.append(model)

model = bert_model()
model.load_weights(model_path3)
models.append(model)

model = bert_model()
model.load_weights(model_path4)
models.append(model)

model = bert_model()
model.load_weights(model_path5)
models.append(model)

In [11]:
len(models)

5

In [12]:
test_predictions = []
for model in models:
    test_predictions.append(model.predict(test_inputs, batch_size=8, verbose=1))

476/476 [==============================] - 20s 42ms/sample


In [13]:
final_predictions = np.mean(test_predictions, axis=0)
df_sub.iloc[:, 1:] = final_predictions
df_sub.to_csv('submission.csv', index=False)

In [14]:




# # model.layers[3].trainable = False
# # x = tf.keras.layers.GlobalAveragePooling1D()(model.layers[3].output[1])
# # x = tf.keras.layers.Dropout(0.2)(x)
# # out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)
# # new_model = tf.keras.models.Model(inputs=model.inputs, outputs=out)
# # new_model.load_weights(model_path)

# model_path = f'../input/bert-bilstm/bert_model_fold_1.h5'
# #print(model_path)
# model = bert_model()
# model.load_weights(model_path)
# model.layers[3].trainable = False
# x = tf.keras.layers.GlobalAveragePooling1D()(model.layers[3].output[1])
# x = tf.keras.layers.Dropout(0.2)(x)
# out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)
# new_model = tf.keras.models.Model(inputs=model.inputs, outputs=out)
# # new_model.load_weights(model_path)
# #model = load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer}) 
# #reloaded_model = tf.keras.experimental.load_from_saved_model('path_to_my_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
# models.append(new_model)



# for i in range(3,15):
#     model_path = f'../input/bert-bilstm/bert_model_fold_{i}.h5'
#     #print(model_path)
#     model = bert_model()
#     model.load_weights(model_path)
#     model.layers[3].trainable = False
#     x = tf.keras.layers.GlobalAveragePooling1D()(model.layers[3].output[1])
#     x = tf.keras.layers.Dropout(0.2)(x)
#     out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)
#     new_model = tf.keras.models.Model(inputs=model.inputs, outputs=out)
# #     new_model.load_weights(model_path)
#     #model = load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer}) 
#     #reloaded_model = tf.keras.experimental.load_from_saved_model('path_to_my_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
#     models.append(new_model)

In [15]:
final_predictions = np.mean(test_predictions, axis=0)
df_sub.iloc[:, 1:] = final_predictions
df_sub.to_csv('submission.csv', index=False)

In [16]:
# test_predictions = [histories[0].test_predictions for i in range(len(histories))]
# test_predictions = [np.average(test_predictions[i], axis=0) for i in range(len(test_predictions))]
# test_predictions = np.mean(test_predictions, axis=0)

# df_sub.iloc[:, 1:] = test_predictions

# df_sub.to_csv('submission.csv', index=False)